In [2]:
%run preprocessing.ipynb

In [3]:
price_only_df['unique_id'] = 'price_only'


In [ ]:
# Standard libraries
import random
import multiprocessing
from math import sqrt

# Data manipulation
import numpy as np
import pandas as pd

# Machine learning
from sklearn.metrics import mean_squared_error
import optuna

# MLForecast
from mlforecast import MLForecast
from mlforecast.auto import (
    AutoMLForecast,
    AutoElasticNet, 
    AutoXGBoost,
    AutoLightGBM,
    AutoCatboost
)
from mlforecast.target_transforms import LocalStandardScaler
from mlforecast.lag_transforms import ExponentiallyWeightedMean, RollingMean

# Visualization
import matplotlib.pyplot as plt
from utilsforecast.plotting import plot_series

# Core forecasting utilities
from coreforecast.scalers import LocalStandardScaler, LocalMinMaxScaler
from coreforecast.grouped_array import GroupedArray

# Set up multiprocessing and seeds
print(multiprocessing.cpu_count())

def set_seeds(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    optuna.logging.set_verbosity(optuna.logging.WARNING)

set_seeds()

In [5]:
# For CatBoost, we only tune subsample parameter due to limited data size
# More parameters could cause overfitting or convergence issues
# subsample: controls the fraction of samples used for each tree building
def catboost_model_params(trial: optuna.Trial):
    return {
        'subsample': trial.suggest_float('subsample', 0.5, 1.0)
    }

def calculate_metrics(actual, predicted):
    """Calculate multiple performance metrics for forecasting evaluation.
    
    Args:
        actual (array-like): The actual/true values
        predicted (array-like): The predicted/forecasted values
        
    Returns:
        tuple: A tuple containing:
            - rmse (float): Root Mean Square Error
            - directional_accuracy (float): Proportion of correctly predicted directions (0-1)
            - turning_point_accuracy (float): Proportion of correctly predicted turning points (0-1)
            - weighted_score (float): Combined score weighing all three metrics equally
    """
    # Convert inputs to numpy arrays and flatten
    actual = np.asarray(actual).flatten()
    predicted = np.asarray(predicted).flatten()

    # Calculate RMSE
    rmse = sqrt(mean_squared_error(actual, predicted))
    
    # Calculate directional accuracy (proportion of correctly predicted up/down movements)
    actual_diff = np.diff(actual)
    pred_diff = np.diff(predicted)
    directional_accuracy = np.mean((actual_diff * pred_diff) > 0)
    
    # Calculate turning point accuracy (proportion of correctly predicted trend changes)
    actual_turns = (actual_diff[:-1] * actual_diff[1:]) < 0  # True when direction changes
    pred_turns = (pred_diff[:-1] * pred_diff[1:]) < 0
    turning_point_accuracy = np.mean(actual_turns == pred_turns)
    
    # Calculate weighted score - lower is better
    # Combines RMSE with penalties for poor directional and turning point accuracy
    weighted_score = (rmse + (1 - directional_accuracy) + (1 - turning_point_accuracy)) / 3
    
    return rmse, directional_accuracy, turning_point_accuracy, weighted_score


In [ ]:
def run_forecasting_pipeline(with_macro_df, horizon=3, step_size=3, n_windows=16):
    """Run an automated machine learning forecasting pipeline with multiple models.
    
    This function implements a complete forecasting workflow including:
    - Train/test splitting
    - Data preprocessing and scaling
    - Model training with cross-validation
    - Prediction generation
    - Performance evaluation and visualization
    
    Args:
        with_macro_df (pd.DataFrame): Input dataframe containing target variable 'y',
            datetime column 'ds', ID column 'unique_id' and optional macro features
        horizon (int, optional): Number of future periods to forecast. Defaults to 3.
        step_size (int, optional): Number of periods between cross-validation windows. Defaults to 3.
        n_windows (int, optional): Number of cross-validation windows. Defaults to 16.
            
    Returns:
        tuple: A tuple containing:
            - auto_mlf (AutoMLForecast): The fitted forecasting model
            - predictions (pd.DataFrame): Future predictions
            - cv_results (dict): Cross-validation results for each model
            - metrics_df (pd.DataFrame): Performance metrics comparison
    """
    # Split data into train and test sets
    # Test set size is determined by number of windows * step size
    train_size = len(with_macro_df) - n_windows * step_size
    train_df = with_macro_df[:train_size].copy()
    test_df = with_macro_df[train_size:].copy()  

    # Basic preprocessing - fill missing values with 0
    processed_df = with_macro_df.copy()
    processed_df.fillna(0)

    # Identify any exogenous (macro) features by excluding standard columns
    macro_features = processed_df.columns.difference(['unique_id', 'ds', 'y'])
    has_exog = len(macro_features) > 0

    # Scale macro features if present using local min-max scaling
    if has_exog:
        scaler = LocalMinMaxScaler()
        
        # First scale training data
        for feature in macro_features:
            train_values = train_df[feature].values
            indptr = np.array([0, len(train_values)])
            grouped_train = GroupedArray(train_values, indptr)
            scaled_train_values = scaler.fit_transform(grouped_train)
            train_df[feature] = scaled_train_values

        # Then scale full dataset using fitted scaler
        for feature in macro_features:
            full_values = processed_df[feature].values
            indptr = np.array([0, len(full_values)])
            grouped_full = GroupedArray(full_values, indptr)
            scaled_full_values = scaler.transform(grouped_full)
            processed_df[feature] = scaled_full_values

    # Initialize dictionary of models to evaluate
    models = {
        'elasticnet': AutoElasticNet(),  # Linear model with L1/L2 regularization
        'xgboost': AutoXGBoost(),        # Gradient boosting with trees
        'lightgbm': AutoLightGBM(),      # Light gradient boosting
        'catboost': AutoCatboost(config = catboost_model_params)  # Categorical boosting
    }

    # Configure automated ML forecasting framework
    auto_mlf = AutoMLForecast(
        models=models,
        freq='M',  # Monthly frequency
        season_length=12,  # Annual seasonality
        fit_config=lambda trial: {
            'static_features': [],
            'dropna': True,
            'keep_last_n': None
        },
        num_threads=12  # Parallel processing
    )

    # Fit models with cross-validation
    print("Performing optimization and cross-validation...")
    auto_mlf.fit(
        train_df,
        n_windows=16,
        h=3,
        num_samples=100,
        step_size=3
    )

    # Generate future prediction dataframe
    print("\nGenerating predictions...")
    any_model = next(iter(auto_mlf.models_.values()))
    future_df = any_model.make_future_dataframe(h=horizon)
    
    # Handle future macro features if present
    if has_exog:
        # Get last known values for each series
        last_dates = with_macro_df.groupby('unique_id')['ds'].max()
        future_values = []
        
        # Create future macro data using last known values
        for idx, row in future_df.iterrows():
            uid = row['unique_id']
            last_known_values = with_macro_df[with_macro_df['unique_id'] == uid].loc[
                with_macro_df['ds'] == last_dates[uid], 
                macro_features
            ].iloc[0]
            
            future_values.append({
                'unique_id': uid,
                'ds': row['ds'],
                **last_known_values
            })
        
        # Scale future macro features
        future_macro_df = pd.DataFrame(future_values)
        for feature in macro_features:
            future_values = future_macro_df[feature].values
            indptr = np.array([0, len(future_values)])
            grouped_future = GroupedArray(future_values, indptr)
            scaled_future_values = scaler.transform(grouped_future)
            future_macro_df[feature] = scaled_future_values
        
        # Generate predictions with exogenous features
        predictions = auto_mlf.predict(horizon, X_df=future_macro_df)
    else:
        # Generate predictions without exogenous features
        predictions = auto_mlf.predict(horizon)

    # Evaluate models using cross-validation
    cv_results = {}
    metrics = {}

    # Loop through each model for evaluation
    for model_name, model in auto_mlf.models_.items():
        # Perform cross-validation on last 48 periods
        cv_df = model.cross_validation(
            df=processed_df,
            n_windows=n_windows,
            h=horizon,
            step_size=step_size,
            static_features=[],
            dropna=True,
            keep_last_n=48,
        )
        cv_results[model_name] = cv_df
        actual = cv_df['y']
        predicted = cv_df[model_name]
        
        # Calculate performance metrics
        rmse, dir_acc, turn_acc, weighted_score = calculate_metrics(actual, predicted)
        metrics[model_name] = {
            'RMSE': rmse,
            'Directional Accuracy': dir_acc,
            'Turning Point Accuracy': turn_acc,
            'Weighted Score': weighted_score
        }

        # Create evaluation plots
        plt.figure(figsize=(15, 6))
        print(f"\nMetrics for {model_name}:")
        print(f"RMSE: {rmse:.4f}")
        print(f"Directional Accuracy: {dir_acc:.4f}")
        print(f"Turning Point Accuracy: {turn_acc:.4f}")
        print(f"Weighted Score: {weighted_score:.4f}")
        
        # Print value ranges for validation
        print(f"\nValue ranges for {model_name}:")
        print("Original data range:", with_macro_df['y'].min(), "-", with_macro_df['y'].max())
        print("Predicted data range:", cv_df[model_name].min(), "-", cv_df[model_name].max())
        print("Time range:", cv_df['ds'].min(), "-", cv_df['ds'].max())

        # Plot actual vs predicted values
        plt.plot(cv_df['ds'], cv_df['y'], 'b.', label='Actual', alpha=0.5)
        for uid in cv_df['unique_id'].unique():
            mask = cv_df['unique_id'] == uid
            plt.plot(cv_df.loc[mask, 'ds'], 
                    cv_df.loc[mask, model_name], 
                    'r-', alpha=0.3)
        
        plt.title(f'Cross-validation results for {model_name} (Last 48 Periods Only)\nWeighted Score: {weighted_score:.4f}')
        plt.ylabel('Value')
        plt.xlabel('Time')
        plt.grid(True)
        
        # Set y-axis limits with padding
        y_min = min(with_macro_df['y'].min(), cv_df[model_name].min())
        y_max = max(with_macro_df['y'].max(), cv_df[model_name].max())
        padding = (y_max - y_min) * 0.1
        plt.ylim(y_min - padding, y_max + padding)
        
        plt.legend(['Actual', 'Predicted'])
        plt.tight_layout()
        plt.show()

    # Create comparison metrics dataframe
    metrics_df = pd.DataFrame(metrics).round(4)
    print("\nModel Comparison Metrics:")
    print(metrics_df)

    # Identify best performing model based on weighted score
    best_model = min(metrics.items(), key=lambda x: x[1]['Weighted Score'])
    print(f"\nBest Model: {best_model[0]} (Weighted Score: {best_model[1]['Weighted Score']:.4f})")

    return auto_mlf, predictions, cv_results, metrics_df

# Run the forecasting pipeline
auto_mlf, predictions, cv_results, metrics_df = run_forecasting_pipeline(with_macro_df, horizon=3, step_size=3)